In [1]:
!pwd
import sys
sys.executable

/c/Users/shingo/jupyter_notebook/tfgpu_py36_work/02_keras_py/experiment/01_code_test


'C:\\Users\\shingo\\Anaconda3\\envs\\tfgpu20\\python.exe'

# コードのテスト
## tf_define_model.py 関連の関数について
- ニューラルネットワーク構築する

In [15]:
import os, sys
current_dir = os.path.dirname(os.path.abspath("__file__"))
path = os.path.join(current_dir, '../../')
sys.path.append(path)
from model import tf_define_model as define_model
from model import tf_pooling as pooling
from model import tf_keras_visualize_cnn, tf_lr_finder, tf_multi_loss, tf_my_callback, tf_my_loss
%reload_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))# デフォルトは75%
%matplotlib inline

from tensorflow import keras
import numpy as np

## GeM2D

In [3]:
from tensorflow import keras
input_tensor = keras.layers.Input(shape=(224, 224, 3))
model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
x = model.output
x = pooling.GeM2D()(x)
x = keras.layers.Dense(10)(x)
model = keras.models.Model(inputs=model.input, outputs=x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
model, orig_model = define_model.get_fine_tuning_model('output_dir'
                                                       , 224,224,3
                                                       , 10
                                                       , 'VGG16'
                                                       , trainable=15
                                                       , fcpool='GeM2D')
model.summary()

----- model_param -----
output_dir = output_dir
img_rows img_cols channels = 224 224 3
num_classes = 10
choice_model trainable = VGG16 15
fcs = []
fcpool = GeM2D
pred_kernel_initializer pred_l2_rate = zeros 0.0001
activation = softmax
gpu_count = 1
skip_bn = True
n_multitask = 1
----- FC_layers -----
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)

## BlurPooling2D

In [12]:
from tensorflow import keras
input_tensor = keras.layers.Input(shape=(224, 224, 3))
model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
x = model.output
x = pooling.BlurPooling2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(10)(x)
model = keras.models.Model(inputs=model.input, outputs=x)

#model = define_model.add_label_injection_layer(model, [str(i)+'_class' for i in range(10)])

model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [6]:
model, orig_model = define_model.get_fine_tuning_model('output_dir'
                                                       , 224,224,3
                                                       , 10
                                                       , 'VGG16'
                                                       , trainable=15
                                                       , fcpool='BlurPooling2D')
model.summary()

----- model_param -----
output_dir = output_dir
img_rows img_cols channels = 224 224 3
num_classes = 10
choice_model trainable = VGG16 15
fcs = []
fcpool = BlurPooling2D
pred_kernel_initializer pred_l2_rate = zeros 0.0001
activation = softmax
gpu_count = 1
skip_bn = True
n_multitask = 1
----- FC_layers -----
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 

In [21]:
import pytest

def _predict_layer(layer, X):
    """単一のレイヤーのModelを作って予測を行う。"""
    if isinstance(X, list):
        inputs = [keras.layers.Input(shape=x.shape[1:]) for x in X]
    else:
        inputs = keras.layers.Input(shape=X.shape[1:])
    model = keras.models.Model(inputs=inputs, outputs=layer(inputs))
    
    model.summary()
    
    outputs = model.predict(X)
    if isinstance(outputs, list):
        assert isinstance(model.output_shape, list)
        for o, os in zip(outputs, model.output_shape):
            assert o.shape == (len(X[0]),) + os[1:]
    else:
        assert outputs.shape == (len(X),) + model.output_shape[1:]
    return outputs

def test_BlurPooling2D():
    X = np.zeros((1, 5, 5, 2))
    X[0, 2, 2, 0] = 1
    y = np.array(
        [
            [
                [[0.09765625, 0.0], [0.58593750, 0.0], [0.09765625, 0.0]],
                [[0.58593750, 0.0], [3.51562500, 0.0], [0.58593750, 0.0]],
                [[0.09765625, 0.0], [0.58593750, 0.0], [0.09765625, 0.0]],
            ]
        ]
    )
    pred = _predict_layer(pooling.BlurPooling2D(taps=5), X)
    
    # pytest.approx: 2つの数値（または2組の数値）が、ある程度の許容範囲内で互いに等しいことを表明
    assert pred == pytest.approx(y, abs=1e-5)
    
    
test_BlurPooling2D()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 5, 5, 2)]         0         
_________________________________________________________________
blur_pooling2d_8 (BlurPoolin (None, 3, 3, 2)           0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [8]:
import sys
sys.path.append(r'C:\Users\shingo\Git\blur-pool-keras')
import tf_blurpool

from tensorflow import keras
input_tensor = keras.layers.Input(shape=(224, 224, 3))
model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
x = model.output
x = tf_blurpool.AverageBlurPooling2D()(x)
x = keras.layers.Dense(10)(x)
model = keras.models.Model(inputs=model.input, outputs=x)

model = define_model.add_label_injection_layer(model, [str(i)+'_class' for i in range(10)])

model.summary()

AttributeError: in converted code:

    C:\Users\shingo\Git\blur-pool-keras\tf_blurpool.py:187 call  *
        x = keras.backend.depthwise_conv2d(x, self.blur_kernel, padding='same', strides=(self.pool_size, self.pool_size))

    AttributeError: module 'tensorflow_core.keras.backend' has no attribute 'depthwise_conv2d'
